In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, cohen_kappa_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import shuffle
# ----------------------------------------------------
# 1. Load and Prepare Dataset
# ----------------------------------------------------
df = pd.read_csv("C:/RHEA/S7/Project/Code/Datasets/BRCA_gene_expression.csv", index_col="Unnamed: 0")

# Encode target variable (cancer classes)
le = LabelEncoder()
df["classes"] = le.fit_transform(df["classes"])  # e.g., 0 = Normal, 1 = Cancer

# Define features and labels
X = df.drop(columns=["classes"])
y = df["classes"]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Training set shape:", X_train.shape)
print("Testing set shape:", X_test.shape)

# ----------------------------------------------------
# 2. Train XGBoost for Feature Importance
# ----------------------------------------------------
xgb_selector = XGBClassifier(
    n_estimators=500, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8, random_state=42,
    use_label_encoder=False, eval_metric="logloss"
)
xgb_selector.fit(X_train, y_train)

# Get feature importances
feature_importances = pd.Series(xgb_selector.feature_importances_, index=X.columns)

# ----------------------------------------------------
# 3. Select Top N Important Genes (e.g., Top 500)
# ----------------------------------------------------
top_features = feature_importances.nlargest(500).index.tolist()

# Filter train/test sets
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

# ----------------------------------------------------
# 4. Train Logistic Regression on Selected Features
# ----------------------------------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_selected)
X_test_scaled = scaler.transform(X_test_selected)

lr_clf = LogisticRegression(solver="liblinear", C=1.0, random_state=42)
lr_clf.fit(X_train_scaled, y_train)

# Evaluate model
y_pred = lr_clf.predict(X_test_scaled)
y_pred_proba = lr_clf.predict_proba(X_test_scaled)[:, 1]

print("\nModel Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_proba))
print("Kappa:", cohen_kappa_score(y_test, y_pred))



ModuleNotFoundError: No module named 'pandas'

In [ ]:
# ----------------------------------------------------
# 5. Predict Cancer/Normal for a New Input
# ----------------------------------------------------
print("\n--- Predicting Cancer Status for New Patient ---")

# Simulated gene expression input (example)
# You need to provide expression values for the same top 500 selected genes
# For demonstration, we’ll generate a random vector similar to mean values of training data
sample_input = X_train_selected.mean().values + np.random.normal(0, 0.1, len(top_features))
sample_df = pd.DataFrame([sample_input], columns=top_features)

# Scale input using the same scaler used for training
sample_scaled = scaler.transform(sample_df)

# Predict
prediction = lr_clf.predict(sample_scaled)[0]
prediction_proba = lr_clf.predict_proba(sample_scaled)[0][1]

# Decode the label
predicted_class = le.inverse_transform([prediction])[0]

print("Predicted Class:", predicted_class)
print("Probability of Cancer:", round(prediction_proba, 3))
